In [ ]:
%%writefile assign11.cpp
#include<bits/stdc++.h>
#include <omp.h>
#include <chrono> 
using namespace std;
using namespace std::chrono;

void swap(int *a, int *b){
    int temp = *a;
    *a = *b;
    *b = temp;
}

void parallelBubble(){
    int n = 1<<16;
    int arr[n];
    cout << "\nParallel Bubble Sort\nNumber of elements : " << n << endl;
    for(int i=0; i<n; i++){
        arr[i] = rand()%n;
    }
    auto start = high_resolution_clock::now();
    for(int i=0; i<n-1; i++){
        int first = i%2;
        #pragma omp parallel for
        for(int j = first; j < n-1; j+=2){
		        if(arr[j] > arr[j+1] ) swap(&arr[j], &arr[j+1]);
	    	}
    }
    auto stop = high_resolution_clock::now();
    cout << "Time taken = " << duration_cast<milliseconds>(stop - start).count() << " ms\n";
}

void seq_Bubble(){
    int n = 1<<16;
    int arr[n];
    cout << "\nSequential Bubble Sort\nNumber of elements : " << n << endl;
    for(int i=0; i<n; i++){
        arr[i] = rand()%n;
    }
    
    auto start = high_resolution_clock::now();
    
    for(int i=0; i<n-1; i++){
        for(int j = i; j < n; j+=2){
		        if(arr[j] > arr[j+1]) swap(&arr[j], &arr[j+1]);
	    	}
    }
    auto stop = high_resolution_clock::now();
    cout << "Time taken = " << duration_cast<milliseconds>(stop - start).count() << " ms\n";
}

void merge(int *arr, int start, int mid, int end){
    int len = (end - start) + 1, temp[len], cur = 0, i = start, j = mid+1;

    while(i <= mid && j <= end){
        if(arr[i] <= arr[j]){
            temp[cur] = arr[i];
            i++;
        }
        else{
            temp[cur] = arr[j];
             j++;
        }
        cur++;
    }
    while(i <= mid){
        temp[cur] = arr[i];
        i++;
        cur++;
    }
    while(j <= end){
        temp[cur] = arr[i];
        j++;
        cur++;
    }
    cur = 0;
    for(i=start; i<=end; i++){
        arr[i] = temp[cur];
        cur++;
    }
}

void merge_sort(int *arr, int start, int end){
    if(start < end){
        int mid = (start + end) / 2;

        #pragma omp parallel sections
        {
            #pragma omp section
            merge_sort(arr, start, mid);
            #pragma omp section
            merge_sort(arr, mid+1, end);
        }
        merge(arr, start, mid, end);
    }
}

void seq_merge_sort(int *arr, int start, int end){
    if(start < end){
        int mid = (start + end) / 2;
        seq_merge_sort(arr, start, mid);
        seq_merge_sort(arr, mid+1, end);
        merge(arr, start, mid, end);
    }
}

void seq_Merge(){
    int n = 1<<16;
    int arr[n];
    cout << "\nSequential Merge Sort\nNumber of elements : " << n << endl;
    for(int i=0; i<n; i++){
        arr[i] = rand()%n;
    }
    auto start = high_resolution_clock::now();

    seq_merge_sort(arr, 0, n-1);

    auto stop = high_resolution_clock::now();
    cout << "Time taken = " << duration_cast<milliseconds>(stop - start).count() << " ms\n";
}

void parallelMerge(){
    int n = 1<<16;
    int arr[n];
    cout << "\nParallel Merge Sort\nNumber of elements : " << n << endl;
    for(int i=0; i<n; i++){
        arr[i] = rand()%n;
    }
    auto start = high_resolution_clock::now();
    merge_sort(arr, 0, n-1);
    auto stop = high_resolution_clock::now();
    cout << "Time taken = " << duration_cast<milliseconds>(stop - start).count() << " ms\n";
}

int main(){
    seq_Bubble();
    parallelBubble();
    seq_Merge();
    parallelMerge();
    return 0;
}

Overwriting assign11.cpp


In [ ]:
%%script bash

g++ assign11.cpp -fopenmp -o assign
./assign


Sequential Bubble Sort
Number of elements : 65536
Time taken = 7656 ms

Parallel Bubble Sort
Number of elements : 65536
Time taken = 10397 ms

Sequential Merge Sort
Number of elements : 65536
Time taken = 12 ms

Parallel Merge Sort
Number of elements : 65536
Time taken = 58 ms
